In [2]:
import csv
#import nltk
from gensim.models import Word2Vec
from gensim.models import FastText

import re


In [6]:
seed = 123

In [7]:
dim = 100
method = "w2v"
# read embddings corpus corpus_embeddings
with open('py_corpus.txt', 'r', encoding='utf-8') as file:
    corpus = file.read()#.lower().replace('\n', ' ')

In [8]:
def dropEmpty(tokens0):
    tokens = []
    for i in range(0, len(tokens0)):
        temp = tokens0[i]
        if temp != []:
            tokens.append(temp)
    return tokens

def listToString(s): 
    
    # initialize an empty string
    str1 = "" 
    
    # traverse in the string  
    for ele in s: 
        str1 += ele  
    
    # return string  
    return str1 

def stringToList(string):
    codeLinesList = []
    for line in string.splitlines():
        codeLinesList.append(line)
    return codeLinesList

def remove_comments(input_str):
    
    input_str = re.sub('#.*', '', input_str)
    input_str = re.sub('""".*"""', '', input_str)              

    doc_reg_1 = r'("""|\'\'\')([\s\S]*?)(\1\s*)(?=class)'
    doc_reg_2 = r'(\s+def\s+.*:\s*)\n(\s*"""|\s*\'\'\')([\s\S]*?)(\2[^\n\S]*)'
    input_str = re.sub(doc_reg_1, '', input_str)
    input_str = re.sub(doc_reg_2, r'\1', input_str)
    
    return input_str
    
def dropHeaders(lines):
    linList = []
    for line in lines:
        if not re.search('import',line):
             if not re.search('from',line):
                  linList.append(line)
    return linList

def dropBlank(tokens0):
    tokens = []
    for i in range(0, len(tokens0)):
        temp = tokens0[i]
        if temp != '':
            tokens.append(temp)
    return tokens

def tokenizeLines(codeLinesList):
	"""
	Tokenize each file line, i.e. entry of code list, based on a specified
    regex.
    ***The regex used is not applicable to every case. One needs to configure
    it based on file inputs.

	Return: list containing code tokens
	"""
    #enalaktika codeLinesList.split() gia na meinoun ta tokens mazi me to punctuation
	codeTokens = []

	for line in codeLinesList:
		templineTokens = re.split('[\.,\[\];:(\s)?\\\\!\t{}"<>+=~*&^%/|\\-\']', line)
		codeTokens.extend(templineTokens)

	return codeTokens


def dataTokenization(corpus, filename, comments, strings, numericals, headers):
    
    allTokens = []
    for i in range(0, len(corpus)):
        stringLines = corpus[i]
        
        if numericals == 0:
            stringLinesNoDigit = re.sub(r"$\d+\W+|\b\d+\b|\W+\d+$", "<numId$>", stringLines) #replace numbers 
        else:
            stringLinesNoDigit = stringLines
        
        if strings == 0:
            stringLinesNoStr = re.sub(r'(["])(?:(?=(\\?))\2.)*?\1', "<strId$>", stringLinesNoDigit) #replace strings       
            stringLinesNoChar = re.sub(r"(['])(?:(?=(\\?))\2.)*?\1", "<strId$>", stringLinesNoStr) #replace chars
        else:
            stringLinesNoChar = stringLinesNoDigit
        
        if comments == 0:
            #remove comments from source code
            linesNoCom = remove_comments(stringLinesNoChar)
        else:
            linesNoCom = stringLinesNoChar
        
        #convert source code from string to list of lines
        lines = stringToList(linesNoCom)
        
        #remove headers
        if headers == 0:
            lines = dropHeaders(lines)
        
        #tokenize lines to list of words
        tokens0 = tokenizeLines(lines)
        
        #remove blank lines
        tokens = dropBlank(tokens0)
        
        #lower case
        for w in range(0, len(tokens)):
            tokens[w] = tokens[w].lower()
         
        allTokens.append(tokens)
        
    
    with open(filename,"w", encoding="utf-8") as f:
        wr = csv.writer(f)
        wr.writerows(allTokens)

In [9]:
def embVectors(dim, method, corpus, comments, strings, numericals, headers): 

    # preprocess corpus
    corpusList = corpus.split("def")  # Splitting from 'def'
    dataTokenization(corpusList, "corpus.csv", comments, strings, numericals, headers)
    
    with open(filename+"_corpus.csv", newline='', encoding='utf-8') as f:
        reader = csv.reader(x.replace('\0', '') for x in f)
        data = list(reader)
    data = dropEmpty(data)
    
    #all_sentences = nltk.sent_tokenize(corpus)
    #all_words = [nltk.word_tokenize(sent) for sent in all_sentences]
    #print("saving")
    #with open('pythontraining_processed_' + mode, 'wb') as fp:
    #    pickle.dump(all_words, fp)
    
    if method == "w2v":
        model = Word2Vec(data, size=dim, window=20, workers=4, min_count=1, sg=1)
        fileEmb = method + '_embeddings.txt'
        model.wv.save_word2vec_format(fileEmb, binary=False)
    elif method == "fast-text":
        model_ted = FastText(size=dim, window=20, min_count=1)
        model_ted.build_vocab(data)
        model_ted.train(sentences=data, epochs=2, sg=1)
        model_ted.save('fasttext.model')
        model_ted = FastText.load('fasttext.model')
        fileEmb = method + '_embeddings.txt'
        model_ted.wv.save_word2vec_format(fileEmb, binary=False)
    
        
    return fileEmb

In [ ]:
fileEmb = embVectors(dim, method, corpus, 1, 0, 0, 0)